

In this tutorial, use the [create_scatter_job()](https://docs.cambercloud.com/docs/python-api/mpi/#create_scatter_job) function to run [Athena++](https://github.com/PrincetonUniversity/athena) code to create parallel MPI simulations of a spherical blast wave.


## Spin-Up MPI cluster, compile, and execute simulation code

First import Camber:



In [4]:
import camber



If needed, clone the Athena repo:



In [6]:
# working directory is where the notebook is located
!git clone https://github.com/PrincetonUniversity/athena.git

fatal: destination path 'athena' already exists and is not an empty directory.




Create an MPI job to compile Athena++. This ensures the code is compiled with the correct MPI environment:



In [9]:
compile_job = camber.mpi.create_job(
    command="cd athena && python configure.py --prob=blast -mpi -hdf5 --hdf5_path=${HDF5_PATH} && make clean && make all -j$(nproc)",
    engine_size="SMALL"
)

In [16]:
# check the status of the job
compile_job.status

'FAILED'


Define the parameters:


In [17]:
# The blast problem begins with a central sphere of material with higher pressure than its surroundings.
# The prat parameter sets the ratio. In this case, four values ranging from a factor of 10 to a factor of 300.
params = {
    "prat": ["10", "30", "100", "300"]
}

In [ ]:
# run scatter jobs using MEDIUM engines
# here we take advantage Athena's functionality for over-writing a parameter in the input file frome the
# command line as well as the ability to set the runtime directory
jobs = camber.mpi.create_scatter_job(
    command_template="mpirun -np 16 athena/bin/athena -i athinput.blast problem/prat={prat} -d run{prat}",
    template_params_grid=params,
    engine_size="MEDIUM"
)

In [18]:
# lets check in on how are jobs are doing
jobs

NameError: name 'jobs' is not defined

In [ ]:
# Prepare a directory for output images
!mkdir output_images

In [ ]:
# import a custom script for reading and plotting the hdf5 outputs, placing images in the output_images directory
# each frame is a 2D slice throught the center of the 3D cartesian grid. This assumes each simulation output
# is in its own runtime directory. We pass the same params dictionary used above for the scatter job.
from plot_output import plot_output
plot_output(params, "prat")

### Visualize Data

In [ ]:
from IPython.display import Video
Video("density.mov")